# 6. Serving
 --------------------------------------------------------------------

Deploy the model created in the previous notebook and predict the output of the data coming from `serving-stream`. The model execution and results are tracked using MLRun.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

## Initialize

Load the project

In [1]:
from mlrun import load_project
from os import path

project_path = path.abspath('conf')
project = load_project(project_path)

In [2]:
WEB_API_USERS = project.params.get('WEB_API_USERS')
serving_stream_path = project.params.get('STREAM_CONFIGS').get('serving-stream').get('path')
inference_stream_path = project.params.get('STREAM_CONFIGS').get('inference-stream').get('path')

### Set the Model

In [3]:
MODEL_NAME = 'my_model'
MODEL_PATH = path.join('/User/examples', project.params.get('PROJECT_BASE_NAME'), 'model.pkl')
print(f'Set model path:  {MODEL_PATH}')

Set model path:  /User/examples/model-deployment-with-streaming/model.pkl


### Add Serving Functions from the MLRun Functions Marketplace <a id="gs-ml-pipeline-add-functions"></a>


Add to your project a real-time model-server function (`serving`) via the [`model_server`](https://github.com/mlrun/functions/tree/master/model_server) MLRun marketplace function.

In [4]:
import nuclio
from mlrun import mount_v3io
project.set_function('hub://model_server:development', 'serving')

### Deploy the serving function

Set the trained model, input stream and inference stream

In [5]:
input_stream = '/'.join(s.strip('/') for s in [WEB_API_USERS, serving_stream_path]) + '@modelserver'
partitions = list(range(0,8))

serving = project.func('serving').apply(mount_v3io())
serving.add_model(MODEL_NAME, MODEL_PATH)
serving.set_envs({'INFERENCE_STREAM' : path.join('users', inference_stream_path) })

serving.add_trigger('serving_stream', nuclio.triggers.V3IOStreamTrigger(url=input_stream, partitions=partitions))
serving.spec.config.pop('spec.triggers.http')

serving.spec.min_replicas = 1

Deploy the serving function

In [6]:
serving_addr = serving.deploy()

> 2020-08-20 11:17:03,730 [info] deploy started
[nuclio] 2020-08-20 11:17:05,861 (info) Build complete
[nuclio] 2020-08-20 11:17:12,996 done creating model-deployment-with-streaming-iguazio-sklearn-server, function address: 3.131.87.251:30742


### Trigger the Serving Function

In [7]:
import requests
import json
import numpy as np

feat = [7,799,47,17,143,1560,7,164,11,810.0,162.0,5.0,782.0]
json = json.dumps({'instances': np.array(feat).reshape(1,-1).tolist()})

resp = requests.post(url=f'{serving_addr}/{MODEL_NAME}/predict', 
                     json = json)
print(resp.status_code)
print(resp.text)

200
[0]


### Save Project

In [8]:
project.save()